# Clasificación de imágenes con métodos clásicos y redes neuronales

*Por Ernesto Ignacio Borbón Martínez, José de Jesús Gutiérrez Aldrete, Luis Felipe Villaseñor Navarrete y Gerardo Villegas Contreras*

Si has dado ya tus primeros pasos en el mundo del aprendizaje automático, tal vez ya pudiste clasificar especies de flores dadas las características de sus pétalos y sépalos. Todo es risa y diversión cuando tienes un conjunto de datos compuesto de observaciones con sus atributos, pero, ¿qué pasa cuando tenemos imágenes? ¿Qué procede para clasificar? 

El objetivo de este trabajo es evaluar el rendimiento de distintos modelos de clasificación tradicionales (lineales y no lineales), junto con el uso de redes neuronales para aprendizaje supervisado, para comparar los dos paradigmas usando imágenes como entrada. Para esto se utilizarán 3 bases de datos distintas, donde la primera son una serie de imágenes de artículos de ropa, la segunda son distintos emojis dibujados a mano y, la tercera es una descripción de terreno de un área superficial del planeta Marte.

Los modelos recibirán dos tipos de entrada. Primero se entrenarán con cada pixel de las imágenes como características, y después se utilizarán **descriptores**. Estos extraerán características de las imágenes. Para evaluar a los modelos se aplicará el método de validación cruzada en todas las pruebas, excepto para las que involucran la base de datos de ropa, puesto que ésta ya es bastante grande. En los modelos de clasificación se revisará la exactitud y la sensibilidad por clase, mientras que en los modelos de regresión se usará el error cuadrado medio.


## Las bases de datos

Para comparar los modelos clásicos de aprendizaje de máquina con las redes neuronales, se utilizaron 3 conjuntos de datos. 

El primero es el conjunto de datos Fashion-MNIST, el cual consiste de 70,000 imágenes (60,000 de entrenamiento y 10,000 de prueba) de 10 diferentes prendas de vestir en escala de grises con un tamaño de 28x28, acompañado de sus etiquetas.

El segundo conjunto de datos consiste en 2470 imágenes binarias de emojis dibujados a mano con un tamaño de 32x32. Los diferentes emojis en la base de datos son caritas enojadas (clase 1) felices (2), tristes (4), sorprendidas (5) y el emoji de popó (clase 3). 

El tercer y último conjunto de datos consta de 2000 secciones de imágenes de mapas de elevación de la superficie de Marte tomadas del sitio de HiRISE. Cada registro en el conjunto de datos contiene cuatro imágenes de un área de 200m x 200m acompañados de un indicador de qué tan navegable es esa zona en una escala del 1 al 4 (1: Muy poco navegable – 4: Muy navegable). La primera imagen contiene las alturas, la segunda es un gradiente, la tercera indica las elevaciones en el área y la cuarta las depresiones.

Puedes encontrar las bases de datos emojis y navegability en https://github.com/ShoyChoy/clasificacion-imagenes.git.


## Los descriptores 

Tanto para los modelos clásicos como para las redes neuronales, los modelos se entrenarán de dos maneras distintas. La primera es usando cada uno de los pixeles como la entrada de los modelos y la segunda es usando descriptores que resuman la información que incluye cada imagen.

Los descriptores utilizados son los siguientes:
* Máximo: Devuelve el valor del pixel con el valor más alto.
* Promedio: Devuelve el promedio de los valores de los pixeles. 
* Oblicuidad: Medida que indica que tan asimétrico es el histograma de los valores de los pixeles.
* Curtosis: Medida que indica que tan alto es el pico al hacer un histograma con los valores de los píxeles.
* Varianza:  Medida que indica que tan dispersos están los valores de los pixeles.
*	Disimilaridad: Medida que se saca de la GLCM (gray-level co-occurrence matrix) que similar a la medida “Contraste”, es alta cuando la región tiene un contraste alto
*	Correlación: Medida que se saca de la GLCM (gray-level co-occurrence matrix) e indica si tiene una estructura lineal.


En el caso de las imágenes de Marte los descriptores de disimilaridad y correlación solo se usan en el mapa de alturas. 


## Procedimientos preliminares

Importación de librerías:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

from scipy.stats import kurtosis, skew
from skimage.feature.texture import greycomatrix, greycoprops

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR

from tqdm.notebook import trange

Estas líneas cargarán los datos.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(fashion_tr_im,fashion_tr_lab),(fashion_test_im,fashion_test_lab) = fashion_mnist.load_data()

In [ ]:
emojis = np.loadtxt("emojis.txt") 
x_emojis = emojis[:,1:]
y_emojis = emojis[:,0]

In [ ]:
inputFile = open('navigability.obj', 'rb')
nav = pickle.load(inputFile)

En los siguientes códigos se convierten las matrices del conjunto fashion en vectores unidimensionales, y pasamos los vectores del conjunto emojis a matrices de 32x32. Esto será útil más adelante, para cumplir con los requisitos de entrada de los modelos.

In [ ]:
fashion_tr_im_rz = []
fashion_test_im_rz = []
for i in range(fashion_tr_im.shape[0]):
  fashion_tr_im_rz.append(fashion_tr_im[i].reshape(784))
for i in range(fashion_test_im.shape[0]):
  fashion_test_im_rz.append(fashion_test_im[i].reshape(784))

In [ ]:
reshaped_x_emo = []
for i in range(x_emojis.shape[0]):
  reshaped_x_emo.append(x_emojis[i,:].reshape(32,32))

## Evaluación de modelos clásicos de aprendizaje supervisado

### Usando pixeles y un modelo clásico lineal de clasificación

Para la base de datos de FASHION se clasificaron con un análisis discriminante lineal (LDA). En el caso de los emojis se decidió emplear un clasificador con máquina de soporte vectorial con kernel lineal (SVC).


#### Fashion

In [ ]:
clfFashion = LinearDiscriminantAnalysis()

In [ ]:
clfFashion.fit(fashion_tr_im_rz, fashion_tr_lab)

LinearDiscriminantAnalysis()

In [ ]:
y_pred_fashion = clfFashion.predict(fashion_test_im_rz)
acc = accuracy_score(fashion_test_lab, y_pred_fashion)
rec = recall_score(fashion_test_lab, y_pred_fashion, average = None)
print('Accuracy: ', acc)
print('Recall: ', rec)

Accuracy:  0.8151
Recall:  [0.777 0.933 0.682 0.846 0.743 0.89  0.558 0.893 0.92  0.909]


#### Emojis

In [ ]:
clfEmoji = SVC(kernel='linear')
kf = KFold(n_splits=10, shuffle = True)

In [ ]:
accs = []
recs = []
for train_index, test_index in kf.split(x_emojis):
  clfEmoji.fit(x_emojis[train_index], y_emojis[train_index])
  y_pred_emoji = clfEmoji.predict(x_emojis[test_index])
  accs.append(accuracy_score(y_emojis[test_index], y_pred_emoji))
  recs.append(recall_score(y_emojis[test_index], y_pred_emoji, average = None))

In [ ]:
np.mean(accs)

0.7842105263157895

In [ ]:
recsA = np.mean(recs, axis = 0)
recsA

array([0.66990187, 0.81129023, 0.9959984 , 0.63558921, 0.78589251])

### Usando pixeles y un modelo clásico no lineal de clasificación


El modelo no lineal utilizado para clasificar los datos de FASHION fue KNN con 5 vecinos, evaluado con las mismas métricas de evaluación que se emplearon para el modelo lineal.

Para la clasificación de los datos de los emojis se modeló con máquina de soporte vectorial con función de base radial (SVC ‘rbf’).


#### Fashion

In [ ]:
clfFashionKNN = KNeighborsClassifier(n_neighbors=5)
clfFashionKNN.fit(fashion_tr_im_rz, fashion_tr_lab)

KNeighborsClassifier()

In [ ]:
y_pred_fashion_rad = clfFashionKNN.predict(fashion_test_im_rz)
acc = accuracy_score(fashion_test_lab, y_pred_fashion_rad)
rec = recall_score(fashion_test_lab, y_pred_fashion_rad, average = None)
print('Accuracy: ', acc)
print('Recall: ', rec)

Accuracy:  0.8554
Recall:  [0.855 0.968 0.819 0.86  0.773 0.822 0.575 0.961 0.953 0.968]


#### Emojis

In [ ]:
clfEmojiRad = SVC(kernel='rbf')

In [ ]:
kf = KFold(n_splits=10, shuffle = True)

In [ ]:
accsRad = []
recsRad = []
for train_index, test_index in kf.split(x_emojis):
  clfEmojiRad.fit(x_emojis[train_index], y_emojis[train_index])
  y_pred_emoji = clfEmojiRad.predict(x_emojis[test_index])
  accsRad.append(accuracy_score(y_emojis[test_index], y_pred_emoji))
  recsRad.append(recall_score(y_emojis[test_index], y_pred_emoji, average = None))

In [ ]:
accsRad

[0.8178137651821862,
 0.8502024291497976,
 0.8461538461538461,
 0.8259109311740891,
 0.8299595141700404,
 0.8218623481781376,
 0.8704453441295547,
 0.8582995951417004,
 0.8502024291497976,
 0.8137651821862348]

In [ ]:
np.mean(accsRad)

0.8384615384615385

In [ ]:
recsARad = np.mean(recsRad, axis = 0)
recsARad

array([0.73262437, 0.88611023, 0.99814815, 0.7072808 , 0.85233049])

### Usando descriptores y un modelo clásico lineal de clasificación


Al emplear los descriptores se reduce el tamaño de la entrada de datos, lo que nos permite utilizar el SVC con base lineal en ambas bases de datos. 


#### Fashion

Primero hay que obtener los descriptores. En el siguiente código estos son almacenados en un dataframe. Hacemos eso tanto para el conjunto de entrenamiento como el de prueba.

In [ ]:
fashdf = pd.DataFrame(columns =['Promedio','Maximo', 'Varianza', 'Sesgo', 'Curtosis', 'Disimilaridad', 'Correlación'])

In [ ]:
for obs in trange(len(fashion_tr_im_rz)):
  new_row = []
  new_row.append(fashion_tr_im_rz[obs].mean())
  new_row.append(fashion_tr_im_rz[obs].max())
  new_row.append(fashion_tr_im_rz[obs].var())
  new_row.append(skew(fashion_tr_im_rz[obs].flatten()))
  new_row.append(kurtosis(fashion_tr_im_rz[obs].flatten()))
  tmp = (fashion_tr_im[obs]-fashion_tr_im[obs].min()).astype(int)
  glcm = greycomatrix(tmp , distances=[5], angles=[0], levels=784, symmetric=True, normed=True)
  new_row.append(greycoprops(glcm, 'dissimilarity')[0, 0])
  new_row.append(greycoprops(glcm, 'correlation')[0, 0])
  fashdf.loc[len(fashdf)] = new_row

  0%|          | 0/60000 [00:00<?, ?it/s]

In [ ]:
fashdf

,Promedio,Maximo,Varianza,Sesgo,Curtosis,Disimilaridad,Correlación
0,97.253827,255.0,10361.681746,0.219186,-1.816807,44.604037,0.674014
1,107.905612,255.0,10166.980887,0.013599,-1.849794,95.836957,0.086848
2,36.558673,255.0,2469.965945,1.253612,1.568225,44.385093,0.164066
3,59.501276,255.0,4205.431121,0.439461,-1.307167,62.456522,0.152527
4,78.044643,255.0,10783.420201,0.653719,-1.488643,99.863354,0.112331
...,...,...,...,...,...,...,...
59995,26.738520,255.0,3053.080863,2.068636,3.329212,36.611801,0.155008
59996,54.542092,255.0,8656.283943,1.247745,-0.294726,75.226708,0.218087
59997,79.778061,255.0,11558.578294,0.722509,-1.361234,104.068323,0.123352
59998,42.742347,255.0,1482.915758,0.701708,1.860904,35.507764,0.061244


In [ ]:
fash_testdf = pd.DataFrame(columns =['Promedio','Maximo', 'Varianza', 'Sesgo', 'Curtosis', 'Disimilaridad', 'Correlación'])

In [ ]:
for obs in trange(len(fashion_test_im_rz)):
  new_row = []
  new_row.append(fashion_test_im_rz[obs].mean())
  new_row.append(fashion_test_im_rz[obs].max())
  new_row.append(fashion_test_im_rz[obs].var())
  new_row.append(skew(fashion_test_im_rz[obs].flatten()))
  new_row.append(kurtosis(fashion_test_im_rz[obs].flatten()))
  tmp = (fashion_test_im[obs]-fashion_test_im[obs].min()).astype(int)
  glcm = greycomatrix(tmp , distances=[5], angles=[0], levels=784, symmetric=True, normed=True)
  new_row.append(greycoprops(glcm, 'dissimilarity')[0, 0])
  new_row.append(greycoprops(glcm, 'correlation')[0, 0])
  fash_testdf.loc[len(fash_testdf)] = new_row

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
fash_testdf

,Promedio,Maximo,Varianza,Sesgo,Curtosis,Disimilaridad,Correlación
0,42.673469,255.0,4719.594908,1.208112,-0.105474,23.850932,0.723251
1,128.818878,255.0,12688.592195,-0.130813,-1.860965,103.470497,0.112237
2,65.714286,255.0,10247.015306,1.009458,-0.840044,99.447205,0.071071
3,45.123724,255.0,5920.845662,1.442453,0.569551,66.751553,0.197086
4,79.917092,254.0,4320.678075,-0.033439,-1.364594,64.594720,0.025670
...,...,...,...,...,...,...,...
9995,76.091837,254.0,8559.241566,0.533298,-1.526896,42.473602,0.651113
9996,30.617347,255.0,3332.511740,1.965041,3.029607,48.475155,0.168121
9997,45.311224,255.0,6005.984772,1.366388,0.189099,54.649068,0.365926
9998,49.970663,255.0,7585.972354,1.259243,-0.284168,80.580745,0.090820


Ahora que tenemos las entradas como queremos, pasamos a entrenar el modelo.

In [ ]:
clfFashion_des = SVC(kernel='linear')

In [ ]:
clfFashion_des.fit(fashdf, fashion_tr_lab)

SVC(kernel='linear')

In [ ]:
y_pred_fashion_des = clfFashion_des.predict(fash_testdf)
acc = accuracy_score(fashion_test_lab, y_pred_fashion_des)
rec = recall_score(fashion_test_lab, y_pred_fashion_des, average = None)
print('Accuracy: ', acc)
print('Recall: ', rec)

Accuracy:  0.5437
Recall:  [0.216 0.844 0.324 0.477 0.525 0.73  0.249 0.776 0.592 0.704]


#### Emojis

De nuevo, comenzamos calculando los descriptores y almacenándolos en un dataframe. Posteriormente entrenamos a los modelos con ellos.

In [ ]:
emodf = pd.DataFrame(columns =['Promedio','Maximo', 'Varianza', 'Sesgo', 'Curtosis', 'Disimilaridad', 'Correlación'])

In [ ]:
for obs in trange(len(x_emojis)):
  new_row = []
  new_row.append(x_emojis[obs].mean())
  new_row.append(x_emojis[obs].max())
  new_row.append(x_emojis[obs].var())
  new_row.append(skew(x_emojis[obs].flatten()))
  new_row.append(kurtosis(x_emojis[obs].flatten()))
  tmp = (reshaped_x_emo[obs]-reshaped_x_emo[obs].min()).astype(int)
  glcm = greycomatrix(tmp , distances=[5], angles=[0], levels=1024, symmetric=True, normed=True)
  new_row.append(greycoprops(glcm, 'dissimilarity')[0, 0])
  new_row.append(greycoprops(glcm, 'correlation')[0, 0])
  emodf.loc[len(emodf)] = new_row

  0%|          | 0/2470 [00:00<?, ?it/s]

In [ ]:
emodf.head()

,Promedio,Maximo,Varianza,Sesgo,Curtosis,Disimilaridad,Correlación
0,0.180664,1.0,0.148025,1.660011,0.755636,0.298611,-0.019352
1,0.191406,1.0,0.154770,1.568823,0.461205,0.327546,-0.057562
2,0.239258,1.0,0.182014,1.222333,-0.505902,0.369213,0.001418
3,0.257812,1.0,0.191345,1.107319,-0.773844,0.391204,-0.009800
4,0.226562,1.0,0.175232,1.306416,-0.293278,0.349537,0.007116


In [ ]:
clfEmoji_des = SVC(kernel='linear')

In [ ]:
kf = KFold(n_splits=10, shuffle = True)

In [ ]:
accs_des = []
recs_des = []
for train_index, test_index in kf.split(emodf):
  clfEmoji_des.fit(emodf.iloc[train_index], y_emojis[train_index])
  y_pred_emoji_des = clfEmoji_des.predict(emodf.iloc[test_index])
  accs_des.append(accuracy_score(y_emojis[test_index], y_pred_emoji_des))
  recs_des.append(recall_score(y_emojis[test_index], y_pred_emoji_des, average = None))

In [ ]:
accs_des

[0.3481781376518219,
 0.30364372469635625,
 0.2874493927125506,
 0.291497975708502,
 0.3117408906882591,
 0.29959514170040485,
 0.30364372469635625,
 0.291497975708502,
 0.3117408906882591,
 0.29959514170040485]

In [ ]:
np.mean(accs_des)

0.3048582995951417

In [ ]:
recsA_des = np.mean(recs_des, axis = 0)
recsA_des

array([0.13187504, 0.66560358, 0.64365362, 0.04672734, 0.03595273])

### Usando descriptores y un modelo clásico no lineal de clasificación

Para terminar con la clasificación de los modelos tradicionales se empleó un clasificador de máquina de soporte vectorial con función de base radial (SVC ‘rbf’).


#### Fashion

In [ ]:
clfFashion_desRad = SVC(kernel='rbf')

In [ ]:
clfFashion_desRad.fit(fashdf, fashion_tr_lab)

SVC()

In [ ]:
y_pred_fashion_desRad = clfFashion_desRad.predict(fash_testdf)
acc = accuracy_score(fashion_test_lab, y_pred_fashion_desRad)
rec = recall_score(fashion_test_lab, y_pred_fashion_desRad, average = None)
print('Accuracy: ', acc)
print('Recall: ', rec)

Accuracy:  0.223
Recall:  [0.    0.143 0.052 0.    0.404 0.63  0.038 0.516 0.002 0.445]


#### Emojis

In [ ]:
clfEmoji_desRad = SVC(kernel='rbf')

In [ ]:
accs_desRad = []
recs_desRad = []
for train_index, test_index in kf.split(emodf):
  clfEmoji_desRad.fit(emodf.iloc[train_index], y_emojis[train_index])
  y_pred_emoji_desRad = clfEmoji_desRad.predict(emodf.iloc[test_index])
  accs_desRad.append(accuracy_score(y_emojis[test_index], y_pred_emoji_desRad))
  recs_desRad.append(recall_score(y_emojis[test_index], y_pred_emoji_desRad, average = None))

In [ ]:
accs_desRad

[0.3441295546558704,
 0.29959514170040485,
 0.31983805668016196,
 0.3157894736842105,
 0.30364372469635625,
 0.3117408906882591,
 0.2874493927125506,
 0.29554655870445345,
 0.3441295546558704,
 0.30364372469635625]

In [ ]:
np.mean(accs_desRad)

0.3125506072874494

In [ ]:
recsA_desRad = np.mean(recs_desRad, axis = 0)
recsA_desRad

array([0.07017549, 0.54198027, 0.59188145, 0.23389001, 0.11461515])

### Usando descriptores y un modelo de regresión clásico lineal con Navegability

Como las clases de este conjunto de datos son ordinales, el problema puede tratarse como uno de regresión. Así, se ajustó un regresor KNN de 3 vecinos, teniendo como variable dependiente el nivel de navegabilidad. Las variables predictoras a usar son los descriptores, que almacenamos en un dataframe.

In [ ]:
navdf = pd.DataFrame(columns =['Navegabilidad', 'Valor_maximo', 'Promedio_1', 'Maximo_1', 'Varianza_1', 'Sesgo_1', 'Curtosis_1', 'Disimilaridad_1', 'Correlacion_1','Promedio_2', 'Maximo_2', 'Varianza_2', 'Sesgo_2', 'Curtosis_2', 'Promedio_3', 'Maximo_3', 'Varianza_3', 'Sesgo_3', 'Curtosis_3','Promedio_4', 'Maximo_4', 'Varianza_4', 'Sesgo_4', 'Curtosis_4'])


In [ ]:
for obs in trange(len(nav)):
  new_row = []
  new_row.append(nav[obs][0])
  new_row.append(nav[obs][1])
  for i in range(4):
    new_row.append(nav[obs][i+2].mean())
    new_row.append(nav[obs][i+2].max())
    new_row.append(nav[obs][i+2].var())
    new_row.append(skew(nav[obs][i+2].flatten()))
    new_row.append(kurtosis(nav[obs][i+2].flatten()))
    if i == 1:
      tmp = (nav[obs][i+2]-nav[obs][i+2].min()).astype(int)
      glcm = greycomatrix(tmp , distances=[5], angles=[0], levels=1024, symmetric=True, normed=True)
      new_row.append(greycoprops(glcm, 'dissimilarity')[0, 0])
      new_row.append(greycoprops(glcm, 'correlation')[0, 0])
  navdf.loc[len(navdf)] = new_row


  0%|          | 0/849 [00:00<?, ?it/s]

In [ ]:
navdf

,Navegabilidad,Valor_maximo,Promedio_1,Maximo_1,Varianza_1,Sesgo_1,Curtosis_1,Disimilaridad_1,Correlacion_1,Promedio_2,Maximo_2,Varianza_2,Sesgo_2,Curtosis_2,Promedio_3,Maximo_3,Varianza_3,Sesgo_3,Curtosis_3,Promedio_4,Maximo_4,Varianza_4,Sesgo_4,Curtosis_4
0,1.0,124.81,63.099059,80.106997,92.669347,0.019764,-1.266744,2.186973,4.410643,1.024927,-0.129390,-0.934882,1.243333,0.061970,0.141823,4.844097,0.292197,4.797127,26.790544,1.630983,10.779272,8.577229,1.714322,1.708571
1,1.0,124.81,99.453294,113.058135,44.811539,-0.462523,-0.370325,1.816601,4.796872,0.737577,0.961884,1.738542,0.716667,-0.064027,0.213717,5.762690,0.756701,4.536075,20.669289,9.052902,22.196431,34.362672,-0.055559,-0.871881
2,2.0,124.81,56.945795,68.061221,49.407381,-0.261596,-1.033085,1.552972,3.272865,0.298558,0.303162,0.851488,0.390000,0.163290,2.498995,13.885132,14.450581,1.365751,0.595322,0.929357,6.077656,2.791653,1.617718,1.241940
3,1.0,124.81,96.545285,113.487168,144.683633,-0.346455,-1.173588,2.416150,5.106043,1.147192,0.023358,-0.758841,1.436667,-0.141662,1.027062,12.330952,7.091965,2.704944,6.299525,8.696092,22.625464,66.604375,0.223827,-1.535839
4,1.0,124.81,96.290995,109.079521,56.771252,-0.252860,-1.039188,1.868154,4.875915,1.100352,0.596716,-0.537479,1.313333,-0.005451,0.692666,10.120684,3.137352,3.006618,8.894896,1.402263,8.742954,5.290531,1.545937,1.176913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,4.0,385.56,363.154837,365.776621,0.951538,-0.548345,0.495836,0.252592,0.726708,0.024668,0.855385,-0.073594,0.000000,1.000000,0.011723,0.358164,0.001961,4.675450,23.596816,0.100547,1.886309,0.072990,3.510237,13.159878
845,4.0,385.56,306.087917,308.530098,1.338021,0.150018,-0.670949,0.264917,0.840152,0.017659,1.142501,1.822501,0.000000,1.000000,0.002465,0.189863,0.000281,8.598207,82.870751,0.002681,0.205354,0.000298,8.416542,78.962465
846,4.0,385.56,291.465587,296.164082,4.467064,-0.069998,-0.597375,0.391972,0.770668,0.018986,0.134659,-0.291454,0.000000,1.000000,0.001311,0.192825,0.000196,11.573879,138.934631,0.005512,0.439170,0.001545,8.290757,72.709255
847,4.0,385.56,279.262810,282.840996,1.201961,0.058604,-0.166869,0.345129,1.097548,0.032992,1.076262,1.782806,0.020000,-0.010101,0.007136,0.622971,0.002104,9.060010,97.193228,0.038506,1.760806,0.027337,6.438416,48.913872


In [ ]:
regNav = KNeighborsRegressor(n_neighbors=3)

In [ ]:
kf = KFold(n_splits=10, shuffle = True)

In [ ]:
x=navdf.iloc[:,1:]
y=navdf["Navegabilidad"]

MSE = []

for train_index, test_index in kf.split(navdf):
  regNav.fit(x.iloc[train_index], y[train_index])
  y_pred_nav = regNav.predict(x.iloc[test_index])
  MSE.append(mean_squared_error(y[test_index], y_pred_nav))
print('MSE:',np.mean(MSE))

MSE: 0.625732959850607


## Clasificación con modelos perceptrón multicapa

### Usando los pixeles de las imágenes

El tipo de red neuronal que se ajustó fue un modelo perceptrón multicapa, estos modelos fueron entrenados con cada pixel de las imágenes, para los datos de Fashion se emplearon 2 capas ocultas con 8 neuronas cada una, mientras que para los datos de emoji se emplearon 2 capas ocultas y 15 neuronas por capa. La función de activación de las capas ocultas es la función ReLu mientras que la de la capa de salida es la función softmax.

#### Fashion

In [ ]:
(fashion_tr_im,fashion_tr_lab),(fashion_test_im,fashion_test_lab) = fashion_mnist.load_data()

fashion_tr_im_rz = []
fashion_test_im_rz = []
for i in range(fashion_tr_im.shape[0]):
  fashion_tr_im_rz.append(fashion_tr_im[i].reshape(784))
for i in range(fashion_test_im.shape[0]):
  fashion_test_im_rz.append(fashion_test_im[i].reshape(784))

In [ ]:
n_features = 784

fashion_tr_lab= pd.get_dummies(fashion_tr_lab).to_numpy()

clf = Sequential()
clf.add(Dense(8, input_dim=n_features, activation='relu'))
clf.add(Dense(8, activation='relu'))
clf.add(Dense(10, activation='softmax')) 

clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])
 
clf.fit(np.array(fashion_tr_im_rz), np.array(fashion_tr_lab), epochs=100, batch_size=5, verbose = 0)

y_pred = np.argmax(clf.predict(np.array(fashion_test_im_rz)), axis = -1)

In [ ]:
np.unique(y_pred)

array([2, 5, 7, 8, 9])

In [ ]:
print('ACC: ', accuracy_score(fashion_test_lab, y_pred))
print('Recall:', recall_score(fashion_test_lab, y_pred, average =None))

ACC:  0.4719
Recall: [0.    0.    0.98  0.    0.    0.906 0.    0.921 0.963 0.949]


#### Emojis

In [ ]:
kf = KFold(n_splits=5, shuffle = True)

accs = []
recs = []
x_emojis = emojis[:,1:]
y_emojis = emojis[:,0]

for train_index, test_index in kf.split(x_emojis):

    x_train = x_emojis[train_index]
    y_train = y_emojis[train_index]
    y_train = pd.get_dummies(y_train).to_numpy()  
    
    clf = Sequential()
    clf.add(Dense(15, input_dim=1024, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(5, activation='softmax'))
    clf.compile(loss='categorical_crossentropy', optimizer='adam')
    clf.fit(x_train, y_train, epochs=100, batch_size=3, verbose=0)    

    x_test = x_emojis[test_index, :]
    y_test = y_emojis[test_index]
    y_pred = np.argmax(clf.predict(x_test), axis = -1) +1

    accs.append(accuracy_score(y_test, y_pred))
    recs.append(recall_score(y_test, y_pred, average=None))

acc = np.mean(accs)
print('ACC = ', acc)

recall = np.mean(recs, axis= 0)

ACC =  0.8133603238866398


In [ ]:
print("Recall", recall)

Recall [0.70876834 0.83245446 0.99405767 0.68172715 0.83451891]


### Usando los descriptores de las imágenes

El último modelo de clasificación que se realizó fue utilizando como datos de entrada los descriptores de las imágenes, para entrenar las redes se emplearon 2 capas ocultas, con 8 neuronas por capa para los datos de Fashion y 15 neuronas por capa para los datos de emojis. La función de activación de las capas ocultas es la función ReLu mientras que la de la capa de salida es la función softmax.

#### Fashion

In [ ]:
(fashion_tr_im,fashion_tr_lab),(fashion_test_im,fashion_test_lab) = fashion_mnist.load_data()

In [ ]:
n_features = len(fashdf.columns)

fashion_tr_lab= pd.get_dummies(fashion_tr_lab).to_numpy()

clf = Sequential()
clf.add(Dense(8, input_dim=n_features, activation='relu'))
clf.add(Dense(8, activation='relu'))
clf.add(Dense(10, activation='softmax')) 

clf.compile(loss='categorical_crossentropy', optimizer='adam')

clf.fit(fashdf, np.array(fashion_tr_lab), epochs=100, batch_size=5, verbose = 0)

y_pred = np.argmax(clf.predict(fash_testdf), axis = -1)

In [ ]:
print('ACC: ', accuracy_score(fashion_test_lab, y_pred))
print('Recall:', recall_score(fashion_test_lab, y_pred, average =None))

ACC:  0.4963
Recall: [0.626 0.887 0.295 0.402 0.    0.754 0.162 0.815 0.445 0.577]


#### Emoji

In [ ]:
kf = KFold(n_splits=5, shuffle = True)

accs = []
recs = []
x_emojis = emodf
y_emojis = emojis[:,0]

features = emodf.columns
n_features = len(features)

for train_index, test_index in kf.split(x_emojis):

    x_train = emodf.iloc[train_index]
    y_train = y_emojis[train_index]
    y_train = pd.get_dummies(y_train).to_numpy()  
    
    clf = Sequential()
    clf.add(Dense(15, input_dim=n_features, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(5, activation='softmax'))
    clf.compile(loss='categorical_crossentropy', optimizer='adam')
    clf.fit(x_train, y_train, epochs=100, batch_size=3, verbose=0)    

    x_test = emodf.iloc[test_index, :]
    y_test = y_emojis[test_index]
    y_pred = np.argmax(clf.predict(x_test), axis = -1) +1

    accs.append(accuracy_score(y_test, y_pred))
    recs.append(recall_score(y_test, y_pred, average=None))

acc = np.mean(accs)
print('ACC = ', acc)

recall = np.mean(recs, axis= 0)

ACC =  0.3417004048582996


In [ ]:
print('Recall', recall)

Recall [0.23211087 0.36764032 0.71569646 0.18852025 0.20248154]


#### Navegability

Para finalizar, para el tercer conjunto de datos se modeló con  6 capas con 15 neuronas por capa. En esta red neuronal se añadieron capas con el propósito de obtener mejores resultados que el modelo de regresión tradicional KNN. En las capas ocultas se utilizaron funciones de activación ReLu mientras que la capa de salida tiene una función de activación lineal.


In [ ]:
kf = KFold(n_splits=5, shuffle = True)

mse = 0

x = navdf.iloc[:,1:]
y = navdf['Navegabilidad']

n_features = len(x.columns)
for train_index, test_index in kf.split(x):

    x_train = x.iloc[train_index,:]
    y_train = y.iloc[train_index]

    clf = Sequential()
    clf.add(Dense(15, input_dim=n_features, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(15, activation='relu'))
    clf.add(Dense(1, activation='linear'))
    clf.compile(loss='MeanSquaredError', optimizer='adam')
    clf.fit(x_train, y_train, epochs=100, batch_size=5, verbose=0)

    x_test = x.iloc[test_index, :]
    y_test = y.iloc[test_index]
    y_pred = clf.predict(x_test) 

    mse += mean_squared_error(y_test, y_pred)

In [ ]:
mse = mse/5
print('MSE = ', mse)

MSE =  0.40251182404460356


## Resumen de los resultados

La siguiente tabla muestra el resumen de las métricas por cada modelo de clasificación para el conjunto de datos Fashion.

![picture](https://drive.google.com/uc?id=1Us2oQJHKpXTw3SAQI1_K2MTj4jDk0HMV)

Análogamente, la tabla de abajo presenta el resumen del desempeño de los modelos para el conjunto Emojis.

![picture](https://drive.google.com/uc?id=1nHFwdAMiFiUg8t9llc0QynNpYZbdmDU7)

Para finalizar, esta tabla contiene los errores cuadrados medios de los dos modelos entrenados para el conjunto Navegability.

![picture](https://drive.google.com/uc?id=1eQGwvP6a_Di6mFRq2-AYQk2SEmr5ZiC2)